## Laboratorio Cassandra - CRUD básico

## Sección 1 - Creación de la conexión

Edita los parámetros de conexión en la celda de abajo.

El formato para la lista ```cassandra_endpoints``` es:
```python
[ '{address_1 or hostname_1}:{port_1}', '{address_2 or hostname_2}:{port_2}', ... ]
```

In [ ]:
cassandra_user = ""
cassandra_password = ""
cassandra_endpoints = []

# BORRA ESTE COMENTARIO Y LA EXCEPCION DE ABAJO Y PON TU CODIGO AQUI
raise NotImplementedError("Borra esta excepcion y pon tu código")

***La celda de abajo es de solo lectura. Cualquier modificación será reemplazada por la celda original al momento de la evaluación.***

El objeto **```session```** que se crea al final de la siguiente celda es el que debes usar para realizar operaciones con cassandra

In [ ]:
from cassandra.cluster import Cluster
from cassandra.connection import DefaultEndPoint
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

cassandra_addresses = [
    cassandra_hostnames_with_port.split(":")[0]
    for cassandra_hostnames_with_port in cassandra_endpoints
]
cassandra_ports = [
    int(cassandra_hostnames_with_port.split(":")[1])
    for cassandra_hostnames_with_port in cassandra_endpoints
]
cassandra_nodes = [
    DefaultEndPoint(
        cassandra_address,
        cassandra_port,
    )
    for cassandra_address, cassandra_port in zip(cassandra_addresses, cassandra_ports)
]
print(
    f"🔗 Connecting to: {[f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes]}"
)
print(
    f"JDBC connection URL: jdbc:cassandra://{','.join([f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes])}"
)

auth_provider = PlainTextAuthProvider(
    username=cassandra_user, password=cassandra_password
)
cluster = Cluster(contact_points=cassandra_nodes, auth_provider=auth_provider)
session = cluster.connect()
session.row_factory = dict_factory
print(f"✅ Connected to cluster: {cluster.metadata.cluster_name}")
print(f"🌐 Nodes found: {len(cluster.metadata.all_hosts())}")

***La celda de abajo es de solo lectura. Cualquier modificación será reemplazada por la celda original al momento de la evaluación.***

A continuación se registra el magic %%cql que podrás usar para realizar consultas directamente en las celdas

**Nota que en el método ```cql``` (es decir, el magic method) usa el mismo objeto ```session``` creado arriba para ejecutar la consulta.**

In [ ]:
%xmode Minimal

import re
import pandas as pd
from IPython import get_ipython
from IPython.display import display, HTML
from IPython.core.magic import register_line_cell_magic
from cassandra.query import SimpleStatement
from cassandra import ConsistencyLevel
from cassandra.query import BatchStatement, BatchType

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.html.use_mathjax = True
pd.options.display.html.border = 1
pd.options.display.html.table_schema = False
pd.options.display.expand_frame_repr = False

css = """
<style>
    .dataframe-container {
        max-height: 300px;
        overflow-y: auto !important;
        display: block !important;
    }
    .dataframe thead th {
        position: sticky;
        top: 0;
        background-color: #1a1a1a !important;
        color: #ffffff !important;
        z-index: 2;
        padding: 8px;
    }
    .dataframe tr:hover {
        background-color: #f5f5f5 !important;
        color: black !important;
    }
</style>
"""
display(HTML(css))

log_queries = True


@register_line_cell_magic
def cql(line, cell=None) -> pd.DataFrame:
    ip = get_ipython()
    user_ns = ip.user_ns

    consistency_level = ConsistencyLevel.ONE
    if cell is not None:
        args = [s for s in line.strip().split(" ") if s.strip()]

        if len(args) > 0:
            consistency_level_line = args.pop(0).strip()
            consistency_level = ConsistencyLevel.name_to_value.get(
                consistency_level_line, None
            )
            if consistency_level is None:
                raise ValueError(
                    f'Consistency level "{consistency_level_line}" is not valid'
                )

    code = cell if cell is not None else line

    query = re.sub(r"/\*.*?\*/", "", code, flags=re.DOTALL)
    query = [re.sub(r"(--|#).*$", "", l).rstrip() for l in query.split("\n")]
    query = "\n".join(query).strip()
    query = query.format(**user_ns)

    delimiter_regex = r"(;|(?:\n+\s*\n))"
    statements = [s for s in re.split(delimiter_regex, query) if len(s) > 0]

    while len(statements) > 0:
        statement_join = [statements.pop(0)]
        while len(statements) > 0 and re.match(delimiter_regex, statements[0]):
            statement_join.append(statements.pop(0))

        if statement_join[0].upper().startswith("BEGIN"):
            while len(statements) > 0 and not statement_join[-1].upper().startswith(
                "APPLY"
            ):
                statement_join.append(statements.pop(0))
                while len(statements) > 0 and re.match(delimiter_regex, statements[0]):
                    statement_join.append(statements.pop(0))

            while len(statements) > 0 and re.match(delimiter_regex, statements[0]):
                statement_join.append(statements.pop(0))

        simple_statement = SimpleStatement(f"{''.join(statement_join)}", consistency_level=consistency_level)
        if log_queries:
            print(f"{simple_statement.query_string.strip()}")

        result = session.execute(simple_statement)
        if result and result.current_rows:
            result_df = pd.DataFrame(list(result))
            user_ns["_"] = result_df
            if cell is not None:
                display(
                    HTML(
                        f'<div class="dataframe-container">{result_df.to_html(classes="dataframe", index=True)}</div>'
                    )
                )
        else:
            user_ns["_"] = result
            if cell is not None:
                print(user_ns["_"])

    if cell is None:
        return user_ns["_"]

## Sección 2 - Creación de keyspace

**Usando el magic %%cql** (es decir con una consulta de CQL)

- Crea un **keyspace** llamado ```laboratorio``` con **replication factor** igual a todos los nodos de cassandra en el datacenter **```dc1```** y **```class: NetworkTopologyStrategy```**

In [ ]:
%%cql QUORUM

-- BORRA ESTE COMENTARIO Y PON TU CQL AQUI

In [ ]:
print("Validando keyspace 'laboratorio' y replication factor...")

**Usando el magic %%cql** (es decir con una consulta de CQL)

- En la **keyspace** ```laboratorio``` crea una tabla llamada ```publicaciones``` con las columnas
    - ```usuario_id```, tipo ***uuid***
    - ```publicacion_id```, tipo ***timeuuid***
    - ```titulo```, tipo ***text***
    - ```contenido```, tipo ***text***
    - ```etiquetas```, tipo ***set<text>***
    - ```metadatos```, tipo ***map<text,text>***,
llave primaria ```(usuario_id, publicacion_id)``` y un ***clustering order*** sobre ```publicacion_id``` decendiente

In [ ]:
%%cql QUORUM

-- BORRA ESTE COMENTARIO Y PON TU CQL AQUI

In [ ]:
print('Validando que la tabla "publicaciones" existe...')

## Sección 3 - Inserción de datos

**Usando el magic %%cql** (es decir con una consulta de CQL)

- Inserta los siguientes registros


| Usuario ID (uuid) | Publicación ID (timeuuid) | Título (text) | Contenido (text) | Etiquetas (set) | Metadatos (map) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| `a1b2c3d4-e5f6-7890-abcd-ef1234567890` | now() | Introducción a NoSQL | Cassandra es una base de datos distribuida. | `{'nosql', 'bigdata'}` | `{'idioma': 'es', 'nivel': 'basico'}` |
| `a1b2c3d4-e5f6-7890-abcd-ef1234567890` | now() | Modelado de Datos | El diseño se basa en las consultas. | `{'cassandra', 'modelado'}` | `{'idioma': 'es', 'prioridad': 'alta'}` |
| `f47ac10b-58cc-4372-a567-0e02b2c3d479` | now() | Guía de Instalación | Pasos para configurar un cluster local. | `{'tutorial', 'linux'}` | `{'os': 'ubuntu', 'version': '4.1'}` |
| `f47ac10b-58cc-4372-a567-0e02b2c3d479` | now() | Tips de Rendimiento | Cómo optimizar el uso de Compactación. | `{'performance'}` | `{'autor': 'expert_c'}` |

In [ ]:
%%cql QUORUM

-- BORRA ESTE COMENTARIO Y PON TU CQL AQUI

In [ ]:
print("Validando datos insertados...")

## Sección 4 - Consulta de datos

**Usando el magic %%cql** (es decir con una consulta de CQL)

- Obten todos los datos insertados en la tabla ```publicaciones``` con usuario_id igual a ```a1b2c3d4-e5f6-7890-abcd-ef1234567890```\
- El resultado debe contener las columnas en el siguiente orden
    - usuario_id, 
    - publicacion_id, 
    - contenido, 
    - etiquetas, 
    - metadatos, 
    - titulo 

In [ ]:
%%cql QUORUM

-- BORRA ESTE COMENTARIO Y PON TU CQL AQUI

In [ ]:
print("Validando resultados de la consulta...")

**Usando el magic %%cql** (es decir con una consulta de CQL)

- Obten todos los datos insertados en la tabla ```publicaciones``` con titulo igual a ```'El diseño se basa en las consultas.'```
- El resultado debe contener las columnas en el siguiente orden
    - usuario_id, 
    - publicacion_id, 
    - contenido, 
    - etiquetas, 
    - metadatos, 
    - titulo 

In [ ]:
%%cql QUORUM

-- BORRA ESTE COMENTARIO Y PON TU CQL AQUI

In [ ]:
print("Validando resultados de la consulta...")

## Sección 4 - Actualización de datos

- Las actualizaciones en cassandra requiren la llave primara completa, es decir la columna de partición y la columan de clustering.

- Ya que hasta ahora los registros insertados han utilizado now() para la llave de clustering, debemos conocer primero el valor de la llave de clustering de los registros a actualizar.

- Escribe en la celda de abajo una consulta para obtener el ```publicacion_id``` de los siguientes registros insertados en la seccion anterior:

| Usuario ID (uuid) | Publicación ID (timeuuid) | Título (text) | Contenido (text) | Etiquetas (set) | Metadatos (map) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| `a1b2c3d4-e5f6-7890-abcd-ef1234567890` | now() | Introducción a NoSQL | Cassandra es una base de datos distribuida. | `{'nosql', 'bigdata'}` | `{'idioma': 'es', 'nivel': 'basico'}` |
| `a1b2c3d4-e5f6-7890-abcd-ef1234567890` | now() | Modelado de Datos | El diseño se basa en las consultas. | `{'cassandra', 'modelado'}` | `{'idioma': 'es', 'prioridad': 'alta'}` |

- En una variable llamada ```nosql_intro_publicacion_id``` guarda el ```publicacion_id``` del primer registro
- En una variable llamada ```modelado_publicacion_id``` guarda el ```publicacion_id``` del primer registro

##### Esta vez, **NO** crees una celda con el magic ```%%cql```

##### **Utiliza el objeto ```session``` creado al principio de este notebook**

##### Las variables deben contener unicamente el ```timeuuid``` en string, es decir, deben ser tipo ```str```
##### Recuerda extraer el valor correspondiente del ```ResultSet``` y ```lists``` o las validaciones marcarán el resultado como incorrecto** 
##### **Valida que ```type(nosql_intro_publicacion_id)==str``` y ```type(modelado_publicacion_id)==str```)**

In [ ]:
# BORRA ESTE COMENTARIO Y LA EXCEPCION DE ABAJO Y PON TU CODIGO AQUI
raise NotImplementedError("Borra esta excepcion y pon tu código")

In [ ]:
print(
    f"Validando publicacion_id de registro con título 'Introducción a NoSQL': {nosql_intro_publicacion_id}"
)
print(
    f"Validando publicacion_id de registro con título 'Modelado de Datos': {modelado_publicacion_id}"
)

- Ahora con los valores obtenidos en las variables ```nosql_intro_publicacion_id``` y ```modelado_publicacion_id```, actualiza los registros:

| Usuario ID (uuid) | Publicación ID (timeuuid) | Título (text) | Contenido (text) | Etiquetas (set) | Metadatos (map) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| `a1b2c3d4-e5f6-7890-abcd-ef1234567890` | now() | Introducción a NoSQL | Cassandra es una base de datos distribuida. | `{'nosql', 'bigdata'}` | `{'idioma': 'es', 'nivel': 'basico'}` |
| `a1b2c3d4-e5f6-7890-abcd-ef1234567890` | now() | Modelado de Datos | El diseño se basa en las consultas. | `{'cassandra', 'modelado'}` | `{'idioma': 'es', 'prioridad': 'alta'}` |

- con los siguientes valores:

| Usuario ID (uuid) | Publicación ID (timeuuid) | Título (text) | Contenido (text) | Etiquetas (set) | Metadatos (map) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| `a1b2c3d4-e5f6-7890-abcd-ef1234567890` | {Sin actualizar} | Consultas en cassandra | Requieren la llave primaria completa (columna de partición y columna de clustering) o permitir filtrado (No recomendado) | {Sin actualizar} | {Sin actualizar} |
| `a1b2c3d4-e5f6-7890-abcd-ef1234567890` | {Sin actualizar} | Actualizaciones es cassandra | Requieren la llave primaria completa (columna de partición y columna de clustering) | {Sin actualizar} | {Sin actualizar} |

- Ten cuidado de acutalizar especificamente los registros en el orden correspondiente. 

Es decir, los títulos deben ser actualizados 
- de 'Introducción a NoSQL' a 'Consultas en cassandra' y
- de 'Modelado de Datos' a 'Actualizaciones es cassandra'

y así sus correspondientes contenidos

##### **Usa el magic %%cql** (es decir con una consulta de CQL)


In [ ]:
%%cql QUORUM

-- BORRA ESTE COMENTARIO Y PON TU CQL AQUI

In [ ]:
print(f"Validando actualizacion de registros...")

## Sección 7 - Eliminación de datos

- Elimina el valor de la columna ```contenido``` de la todos los registros con ```usuario_id``` igual a ```a1b2c3d4-e5f6-7890-abcd-ef1234567890```. 

- Puedes usar de nuevo las variables creadas anteriormente ```nosql_intro_publicacion_id``` y ```modelado_publicacion_id``` ya que sus valores siguen apuntando a los ```publicacion_id``` correspondientes

- Como referencia, los contenidos a eliminar estan tachados en la tabla de abajo

| Usuario ID (uuid) | Publicación ID (timeuuid) | Título (text) | Contenido (text) | Etiquetas (set) | Metadatos (map) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| `f47ac10b-58cc-4372-a567-0e02b2c3d479` | now() | Guía de Instalación | ~~Pasos para configurar un cluster local.~~ | `{'tutorial', 'linux'}` | `{'os': 'ubuntu', 'version': '4.1'}` |
| `f47ac10b-58cc-4372-a567-0e02b2c3d479` | now() | Tips de Rendimiento | ~~Cómo optimizar el uso de Compactación.~~ | `{'performance'}` | `{'autor': 'expert_c'}` |

##### **Usa el magic %%cql** (es decir con una consulta de CQL)

In [ ]:
%%cql QUORUM

-- BORRA ESTE COMENTARIO Y PON TU CQL AQUI

In [ ]:
print(f"Validando eliminación de campo 'contenido' de registros actualizados")

- Ahora elimina por completo ambos registros con ```usuario_id``` igual a ```a1b2c3d4-e5f6-7890-abcd-ef1234567890```

In [ ]:
%%cql QUORUM

-- BORRA ESTE COMENTARIO Y PON TU CQL AQUI

In [ ]:
print(f"Validando que los registros fueron eliminados correctamente...")